# 📑 Sellout Data Pipeline Documentation (Bronze to Silver)

This documentation describes the eight notebooks that constitute the **Sellout Data Pipeline**, transforming raw sellout reports (**Bronze**) into a clean, mastered sales and customer dataset (**Silver**).

The pipeline standardizes data, enriches records using external APIs (Google Address Validation), performs entity resolution, and assigns stable master IDs.

| Stage | Output Table | Notebook | Key Actions |
| :--- | :--- | :--- | :--- |
| **Bronze - Silver 01** | `01_sellout_consolidated` | **1/8** | Dynamic Union of all Wholesaler Tables. |
| **Silver 01 - Silver 02** | `02_sellout_consolidated` | **2/8** | Standardize Date, Country (to ISO codes), and VAT number. |
| **Silver 02 - Silver 03 (Prep)** | `03_sellout_customers` | **3/8** | Extract distinct Customer Master records, generate Unique ID, and prepare API payload. |
| **Silver 02 - Silver 03 (Sales)** | `03_sellout_sales` | **4/8** | Enrich sales facts with material/customer master data (KNMT, C4S), calculate UoM conversions to Kg/L. |
| **Silver 03 (API Filter)** | `03_sellout_filtered_place_id` | **5/8** | Filter addresses for lookup, check history, and apply **API call limits**. |
| **Silver 03 (API Execute)** | `google_api_mapping_table` | **6/8** | Execute Google Address Validation API and persist results using a **Delta MERGE**. |
| **Silver 03 (Enrich)** | `03_sellout_with_place_id` | **7/8** | Join full customer data with mapping table to append the **Place ID**. |
| **Silver 03 (Master)** | `031_sellout_customers_int_matched` | **8/8** | Deduplicate customers using the Place ID and assign a stable **Master Customer ID**. |

---

## 1. Data Consolidation and Standardization (Notebooks 1 & 2)

### 1/8: `01_sellout_consolidated`
* **Goal:** Combine all separate bronze tables (one per wholesaler) into a single, unified dataset.
* **Logic:** Dynamically scans the `$bronze\_base\_table\_path` for tables and reads them into a single DataFrame using **`unionByName(allowMissingColumns=True)`**. This handles potential schema drift.
* **Output:** `d_dnst_020_silver.silver_01.01_sellout_consolidated`

### 2/8: `02_sellout_consolidated`
* **Goal:** Clean and normalize critical fields across the unified sellout data.
* **Logic:**
    * **Date Parsing:** Converts diverse string formats into a standardized `DateType` column.
    * **Country:** Normalizes country names (e.g., 'België') into standard **ISO 2-digit codes** (BE, FR, etc.).
    * **VAT Number:** Cleans non-digit characters and prefixes the `vat_number` with the normalized country code.

---

## 2. Sales Fact and Master Data Enrichment (Notebooks 3 & 4)

### 3/8: `03_sellout_customers`
* **Goal:** Create a list of **unique customer entities** for master data management and geocoding preparation.
* **Logic:** Selects distinct customer-identifying columns, generates a permanent **`unique_id`** using hashing, and calculates the address string for API lookup.

### 4/8: `03_sellout_sales`
* **Goal:** Enrich sales records and standardize the sales quantity to Kilograms/Liters (Kg/L).
* **Logic:**
    * **Enrichment:** Joins with C4S and **KNMT** to derive the standard SAP MATNR.
    * **UoM Conversion:** Uses `010_uom_conversion` and **MARM** (`UMREZ`/`UMREN` factors) to calculate the final normalized sales quantity: **$KgL = total\_amount \times \frac{UMREZ}{UMREN}$**.

---

## 3. Geocoding and Customer Mastering (Notebooks 5, 6, 7 & 8)

### 5/8: `03_sellout_filtered_place_id`
* **Goal:** Filter customers needing geocoding and manage API quota.
* **Logic:** Compares the customer list against the mapping table to find missing `place_id` values. Queries history to calculate `api_calls_left` and **limits the result** to stay under the monthly quota.
* **Dependency:** Requires API Key (`google-maps-api-key`).

### 6/8: `google_api_mapping_table`
* **Goal:** Execute the Google Address Validation API and store results.
* **Logic:** Executes the API for the limited list. Results are inserted or updated using a **Delta MERGE statement** based on the address payload, preventing redundant API calls.

### 7/8: `03_sellout_with_place_id`
* **Goal:** Integrate the **Place ID** into the full customer master dataset.
* **Logic:** Left joins the full customer table (`03_sellout_customers`) with the updated `google_api_mapping_table` on the payload keys to append the definitive `place_id`.

### 8/8: `031_sellout_customers_int_matched`
* **Goal:** Deduplicate records and assign a stable, unified master ID.
* **Logic:** Runs the **`apply_matching`** utility using tiered rules:
    * **High Confidence:** Exact Matches on `customer_number`, `vat_number`, and **`place_id`**.
    * **Fallback:** **Fuzzy Match** (Levenshtein score) on name/address, constrained by exact postal code match.
    * The `assign_unified_id` function creates transitive match groups and assigns the persistent **`dim_sellout_cust_id`**.